<a href="https://colab.research.google.com/github/aptr288/Violation_prediction/blob/master/Violation_prediction_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Violation predition
If given number which is flagged by scan tool to be potentially a credit card number our model determines if the number is falsely assumed to be a credit card number or its really confidential data which got exposed and need to be remediated.


In [0]:
#importing required libraries  
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [172]:
#loading data from google drive
os.getcwd()
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/Violation_Prediction/")
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['violation_data.csv', 'violation_data.txt', 'Violation_prediction.ipynb']

In [0]:
import pandas as pd
df = pd.read_csv('violation_data.txt', sep=",")

In [174]:
print(df.head())

       Credit_card              Label
0  379475120338634  Needs Remediation
1  344066675245878  Needs Remediation
2  373505714729768  Needs Remediation
3  376582856800761  Needs Remediation
4  348898329695769  Needs Remediation


In [175]:
print(df.shape)

(269, 2)


# Creating features from different type of inputs

In [0]:
#Checks if given number only contains numricals
df['IsDigit'] = df['Credit_card'].apply(lambda x: 1 if x.isdigit() else 0)

In [178]:
#check if the credit card number is part of decimal number 
def isdecimal(value):
  if re.match(r'\d+\.\d+', value) is None:
    return 0 
  else:
    return 1
print(isdecimal('0.01000'))

1


In [0]:
df['Is_containing_decimal'] = df['Credit_card'].apply(lambda x: isdecimal(x))

In [180]:
#credit number is part of random numbers with length greater than 16
def card_num_part_of_random_num(x):
  if (x.isdigit() and len(x) > 16 ) :
    return 1
  else:
    return 0
print(card_num_part_of_random_num('434444444444444444444444'))

1


In [0]:
df['Num_part_of_random_num'] = df['Credit_card'].apply(lambda x: card_num_part_of_random_num(x))

In [182]:
#Checks if the Credit card is part of text or column has text 
def num_part_of_text(value):
  # print(re.match(r'.*[a-zA-Z]+.*', value))
  if re.match(r'.*[a-zA-Z]+.*', value) is None:
    return 0 
  else:
    return 1
print(num_part_of_text('a'))

1


In [0]:
df['Num_part_of_text'] = df['Credit_card'].apply(lambda x: num_part_of_text(x))

In [184]:
#checks if a given credit card found valid but some of elements are repeating oddly the count of repeatation can be given
def num_repeating(value, given_count): 
    for i in range(len(value)): 
        cur_count = 1
        for j in range(i + 1, len(value)): 
      
            if (value[i] != value[j]): 
                break
            cur_count += 1
        if cur_count >= given_count : 
            return 1
    return 0
print(num_repeating("aanaabbbaaaa", 4))

1


In [0]:
df['Num_repeating'] = df['Credit_card'].apply(lambda x: num_repeating(x, 5))

In [186]:
df.head()

,Credit_card,Label,IsDigit,Is_containing_decimal,Num_part_of_random_num,Num_part_of_text,Num_repeating
0,379475120338634,Needs Remediation,1,0,0,0,0
1,344066675245878,Needs Remediation,1,0,0,0,0
2,373505714729768,Needs Remediation,1,0,0,0,0
3,376582856800761,Needs Remediation,1,0,0,0,0
4,348898329695769,Needs Remediation,1,0,0,0,0


In [0]:
#converting the labels to numaricals 
df['Label'] = df['Label'].apply(lambda x: 1 if x == "Needs Remediation" else 0)

In [0]:
#preparing train test data 
y =df['Label']
X = df.drop(['Label'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = X_train.drop(['Credit_card'], axis=1)


In [0]:
#decision tree model 
destree = tree.DecisionTreeClassifier()
destree.fit(X_train, y_train)
ydecpred = destree.predict(X_test.drop(['Credit_card'], axis=1))

# Test the results of model

In [203]:
print(X_test.iloc[0:5],end="\n")
print("======================================\n")
print("The predicted results for above features")
print(ydecpred[0:5])

                           Credit_card  IsDigit  Is_containing_decimal  \
217                  0.038903543758156        0                      1   
203  XXXXX38600441111111111XXXXX545115        0                      0   
30                     348243304386185        1                      0   
264              419298345968000278787        1                      0   
100                   6011791198486274        1                      0   

     Num_part_of_random_num  Num_part_of_text  Num_repeating  
217                       0                 0              0  
203                       0                 1              1  
30                        0                 0              0  
264                       1                 0              0  
100                       0                 0              0  

The predicted results for above features
[0 0 1 0 1]


1.   The first record is CC number wrongly found in after decimal point so predicted as **false positive**
2.   The second record is CC number wrongly found in among delinquency count or between text so predicted as **false positive**
3. The third record seems to be legitimate credit card finding which **Needs remediation**.


